In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp

from astropy.io import ascii,fits

In [4]:
Be_dat = '/Users/sean/Desktop/MLI_data/beflight.dat'

mtable_out = '/Users/sean/Desktop/MLI_data/fractionalBe.mod'

def make_atten_grid(Be_lowfrac = 0.5, Be_highfrac = 1.0, Be_step = 0.02, E_low=1.0, E_high=97.017, E_bins=1000, Be_dat=Be_dat):
    # mass per cross section ie thickness
    tau = 110e-4
    rho = {'Be': [1.848,99.145], 'BeO': [3.01,0.7], 'Al': [2.6989, 0.04],\
          'C': [2.0,0.01], 'Fe': [7.874,0.08], 'Si': [2.33, 0.02]}
    
    t = 0
    for element in rho:
        t = t + (rho[element][0]*rho[element][1]/100)
        
    print(t)
    t = t*tau
    
    # Interpolate the cross-section of MLI2
    Be_table = np.array(np.loadtxt(Be_dat)).T
    interp_Be = interp.interp1d(Be_table[0]*1000, Be_table[1])
    
    # Make arrays of energy and fraction of thickness
    E_grid = np.logspace(np.log10(E_low), np.log10(E_high), num = E_bins + 1)
    energ_lo = E_grid[:-1]
    energ_hi = E_grid[1:]
    
    frac_grid = np.arange(Be_lowfrac, Be_highfrac + Be_step, Be_step)
    
    # Produce the grid of transmission ratio. This is the (fractional transmission/full transmission)
    atten_grid = np.array([[(((1-f)*np.exp(t*interp_Be(E))) + f) for E in (energ_lo + (energ_hi - energ_lo)/2)] for f in frac_grid])
    
    return (energ_lo, energ_hi), frac_grid, atten_grid
    
    

In [5]:
(energ_lo, energ_hi), frac_grid, atten_grid = make_atten_grid()    
    
primary_hdr = fits.Header()
primary_hdr['HDUCLASS'] = 'OGIP'
primary_hdr['HDUDOC'] = 'OGIP/92-009'
primary_hdr['HDUCLAS1'] = 'XSPEC TABLE MODEL'
primary_hdr['HDUVERS1'] = '1.0.0'
primary_hdr['MODLNAME'] = 'fractionalBe'
primary_hdr['MODLUNIT'] = 'None'
primary_hdr['REDSHIFT'] = False
primary_hdr['ADDMODEL'] = False
primary_hdu = fits.PrimaryHDU(header=primary_hdr)

param_hdr = fits.Header()
param_hdr['HDUCLASS'] = 'OGIP'
param_hdr['EXTNAME'] = 'PARAMETERS'
param_hdr['HDUCLAS1'] = 'XSPEC TABLE MODEL'
param_hdr['HDUCLAS2'] = 'PARAMETERS'
param_hdr['HDUVERS1'] = '1.0.0'
param_hdr['NINTPARM'] = 1
param_hdr['NADDPARM'] = 0
c1 = fits.Column(name='NAME', array=['Be fraction'], format='12A')
c2 = fits.Column(name='METHOD', array=[0], format='J')
c3 = fits.Column(name='INITIAL', array=[0.8], format='E')
c4 = fits.Column(name='DELTA', array=[frac_grid[1]-frac_grid[0]], format='E')
c5 = fits.Column(name='MINIMUM', array=[np.min(frac_grid)], format='E')
c6 = fits.Column(name='BOTTOM', array=[np.min(frac_grid)], format='E')
c7 = fits.Column(name='TOP', array=[np.max(frac_grid)], format='E')
c8 = fits.Column(name='MAXIMUM', array=[np.max(frac_grid)], format='E')
c9 = fits.Column(name='NUMBVALS', array=[len(frac_grid)], format='J')
c10 = fits.Column(name='VALUE', array=[frac_grid], format=str(len(frac_grid)) + 'E')
param_hdu = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10], header = param_hdr)

energy_hdr = fits.Header()
energy_hdr['HDUCLASS'] = 'OGIP'
energy_hdr['EXTNAME'] = 'ENERGIES'
energy_hdr['HDUCLAS1'] = 'XSPEC TABLE MODEL'
energy_hdr['HDUCLAS2'] = 'ENERGIES'
energy_hdr['HDUVERS1'] = '1.0.0'
c1 = fits.Column(name='ENERG_LO', array=np.array(energ_lo), format='E')
c2 = fits.Column(name='ENERG_HI', array=np.array(energ_hi), format='E')
energy_hdu = fits.BinTableHDU.from_columns([c1,c2], header=energy_hdr)

spec_hdr = fits.Header()
spec_hdr['HDUCLASS'] = 'OGIP'
spec_hdr['EXTNAME'] = 'SPECTRA'
spec_hdr['HDUCLAS1'] = 'XSPEC TABLE MODEL'
spec_hdr['HDUCLAS2'] = 'MODEL SPECTRA'
spec_hdr['HDUVERS1'] = '1.0.0'
c1 = fits.Column(name='PARAMVAL', array=frac_grid, format='E')
c2 = fits.Column(name='INTPSPEC', array=atten_grid, format=str(len(energ_lo)) + 'E')
spec_hdu = fits.BinTableHDU.from_columns([c1,c2], header=spec_hdr)

hdul = fits.HDUList([primary_hdu, param_hdu, energy_hdu, spec_hdu])
hdul.writeto(mtable_out, overwrite=True)



1.86131436
